In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
from keras.models import Model
from tensorflow import keras
import pandas as pd
from os import walk, listdir
import os 

In [2]:
categories = pd.read_csv("category_mapping.csv", sep=";")

In [3]:
categories["ID_str"] = categories["ID"].apply(lambda x: str(x).zfill(3))

In [4]:
categories_to_ID = dict(zip(categories["ID_str"],categories["Name"]))

In [5]:
data_path = "./data/train_frames_sample"
output_path = "./data/train_mobilenet_features/"
video_categories = listdir(data_path)
image_files = []
for category in video_categories:
    category_path = data_path+"/"+category
    image_files_category = listdir(category_path)
    image_files = image_files+image_files_category

In [6]:
len(video_categories)

4

In [7]:
dataset = pd.DataFrame()

In [8]:
dataset["frames_files"] = image_files

In [9]:
dataset["category_ID"] = dataset["frames_files"].apply(lambda x: x[:3])

In [10]:
dataset["category_name"] = dataset["category_ID"].apply(lambda x: categories_to_ID[x])

In [11]:
dataset["person_ID"] = dataset["frames_files"].apply(lambda x: x[4:7])
dataset["person_ID"] = dataset["frames_files"].apply(lambda x: x[4:7])

In [12]:
dataset["video_ID"] = dataset["frames_files"].apply(lambda x: x[:11])

In [13]:
dataset["video_path"] = data_path+"/"+dataset["category_ID"]+"/"+dataset["frames_files"]

In [14]:
dataset["frame_ID"] = dataset["frames_files"].apply(lambda x: int(x.split("frame_")[1].split(".")[0]))

In [15]:
MAX_SEQ_LENGTH = dataset["frame_ID"].max()+1
print(MAX_SEQ_LENGTH)

212


# Predict 

In [16]:
CLASS_NAMES =["Sweet milk", "Argentina", "Barbecue", "Thanks"]
MODEL_PATH = "./trained_models/signs_model_efficientnetv2-s_v1"


In [17]:
IMG_SIZE = (224, 224)
IMG_SHAPE = IMG_SIZE + (3,)

model_cnn_features = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


In [18]:
def load_model(model_path):
    model_keras = tf.keras.models.load_model(model_path)
    return model_keras


#model = load_model(MODEL_PATH)
#XX = model.input 
#YY = model.layers[0].output
#model_cnn_features = Model(XX, YY)

In [19]:
width = model_cnn_features.input_shape[1]
height = model_cnn_features.input_shape[2]

In [20]:
dataset_tf = tf.keras.preprocessing.image_dataset_from_directory(data_path, image_size=(width, height))

Found 26300 files belonging to 4 classes.


In [21]:
dataset_predictions = model_cnn_features.predict(dataset_tf) 

ResourceExhaustedError: Graph execution error:

Detected at node 'mobilenetv2_1.00_224/block_1_expand_BN/FusedBatchNormV3' defined at (most recent call last):
    File "C:\Users\lucas\anaconda3\envs\signem\lib\runpy.py", line 192, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\lucas\anaconda3\envs\signem\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "C:\Users\lucas\anaconda3\envs\signem\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\lucas\anaconda3\envs\signem\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\lucas\anaconda3\envs\signem\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\lucas\anaconda3\envs\signem\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    Fil

In [ ]:
dataset_predictions.shape

In [ ]:
total_videos = len(dataset["video_ID"].unique())
i=1
for video_ID in dataset["video_ID"].unique():
    video_frames = dataset[dataset["video_ID"]==video_ID]
    category_ID = video_frames["category_ID"].unique()[0]
    number_of_frames_in_video = len(video_frames["frames_files"].unique())
    path_to_video = output_path+category_ID+"/"
    dataset_video_index = list(dataset[dataset["video_ID"]==video_ID].index)
    print(f"Predicting video: {i}/{total_videos} length: {len(dataset_video_index)}" )
    predictions_video = dataset_predictions[dataset_video_index] 
    zeros_to_pad = MAX_SEQ_LENGTH-predictions_video.shape[0]
    predictions_video = np.pad(predictions_video,  [(0, zeros_to_pad), (0, 0)])
    if not os.path.exists(path_to_video):
        os.makedirs(path_to_video)
    np.savez_compressed(path_to_video+video_ID+'.npz', predictions_video)
    i+=1 

In [ ]:
def frame_from_jpeg(path_jpeg):
    img = Image.open(path_jpeg).resize((width, height), Image.ANTIALIAS)
    img = np.array(img)
    return img

In [ ]:
"""
for video_ID in dataset["video_ID"].unique()[111:]:
    video_frames = dataset[dataset["video_ID"]==video_ID]
    category_ID = video_frames["category_ID"].unique()[0]
    number_of_frames_in_video = len(video_frames["frames_files"].unique())
    path_to_video = "./data/train_cnn_features/"+category_ID+"/"
    dataset_video_index = list(dataset[dataset["video_ID"]==video_ID].index)
    frames_index = list(dataset[dataset["video_ID"]==video_ID].reset_index(drop=True).index)
    print(f"Predicting video: {video_ID} length: {len(dataset_video_index)}" )
    
    video_frames_jpeg = np.empty((number_of_frames_in_video, width, height, 3), dtype=float)
    i = 0
    for video in video_frames["video_path"].unique():
        print(f"{i+1}/{number_of_frames_in_video}")
        video_frames_jpeg[i] = frame_from_jpeg(video)
        i+=1 
    predictions_video = model_cnn_features.predict(video_frames_jpeg) 
    zeros_to_pad = MAX_SEQ_LENGTH-predictions_video.shape[0]
    predictions_video = np.pad(predictions_video,  [(0, zeros_to_pad), (0, 0)])
    if not os.path.exists(path_to_video):
        os.makedirs(path_to_video)
    np.savez_compressed(path_to_video+video_ID+'.npz', predictions_video)
"""